In [95]:
#Set up workspace
import numpy as np
import pandas as pd

In [96]:
from scipy.optimize import minimize, rosen, rosen_der

In [16]:
#Read in data - all active Boston Condos as of January 2018 + all sold Boston condos from 2017
bc_all_2018 = pd.read_csv("BC_sold_active.csv")

#Exclude BR's greater than 4
bc_all = bc_all_2018[bc_all_2018['BEDS']<=4]

Let's create a nested dictionary, with the keys as the number of bedrooms nesting dictionaries of the relevant values for each bedroom count.

In [28]:
#Define keys for dictionary
br_keys = list(set(bc_all['BEDS']))

In [ ]:
#Set an interest rate (for the present value calculation)
int_rate = 0.07

In [59]:
br_dicts = []
for br in br_keys:
    br_df_sold = bc_all.loc[(bc_all['BEDS'] == br) & (bc_all['STATUS']=='SLD')]
    br_df_act = bc_all.loc[(bc_all['BEDS'] == br) & bc_all['STATUS'].isin(['ACT','BOM'])]
    
    #Sold info
    sold_count = len(br_df_sold)
    sold_price = np.mean(br_df_sold['SOLDPRICE'])
    sold_sf = np.mean(br_df_sold['SQFT'])
    sold_ppsf = sold_price/sold_sf
    
    #Act info
    act_count = len(br_df_act)
    act_price = np.mean(br_df_act['LISTPRICE'])
    act_sf = np.mean(br_df_act['SQFT'])
    act_ppsf = act_price/act_sf
    
    #Meta Info
    sales_pm = sold_count/12
    MOS = act_count/sales_pm
    pv_ppsf = np.pv(int_rate/12,MOS,0,-sold_ppsf)
    
    br_dict = {'Sold_Count':sold_count,'Sold_Price': sold_price, 'Sold_SF':sold_sf,
               'Sold_PPSF':sold_ppsf,'Act_Count':act_count,'Act_Price':act_price,
               'Act_SF':act_sf,'Act_PPSF':act_ppsf,'Sales_PM':sales_pm,'MOS':MOS,
               'PV_PPSF':pv_ppsf}
    
    br_dicts.append(br_dict)
     
condo_info = dict(zip(br_keys,br_dicts))

Now that we have all of the relevant market information, we can set up the optimizer

In [64]:
lot_area = 10000
FAR = 8
developable_area = lot_area*FAR

In [98]:
condo_info[2]

{'Act_Count': 203,
 'Act_PPSF': 1037.897354960851,
 'Act_Price': 1412424.9162561577,
 'Act_SF': 1360.8522167487686,
 'MOS': 1.0831480658070254,
 'PV_PPSF': 709.6740609867494,
 'Sales_PM': 187.41666666666666,
 'Sold_Count': 2249,
 'Sold_PPSF': 714.1591253244807,
 'Sold_Price': 787426.6438417074,
 'Sold_SF': 1102.5927078701645}

In [81]:
br_sf_vector = np.array([condo_info[br]['Sold_SF'] for br in br_keys])

In [86]:
sales_pm_vector = np.array([condo_info[br]['Sales_PM'] for br in br_keys])

In [85]:
#What's the equation we want to maximize (minimize)?

#[new_br_count_vector]*[br_sf_vector]*[new_pv_ppsf_vector]

new_br_count = np.array([])
br_sf_vector = np.array([condo_info[br]['Sold_SF'] for br in br_keys]) 
total_sf_vector = br_sf_vector*new_br_count

sales_pm_vector = np.array([condo_info[br]['Sales_PM'] for br in br_keys])
active_counts_vector = np.array([condo_info[br]['Act_Count'] for br in br_keys]) 
new_mos = (active_counts_vector+new_br_count)/sales_pm_vector
sold_ppsf_vector = np.array([condo_info[br]['Sold_PPSF'] for br in br_keys]) 

new_pv_ppsf_vector = np.array([np.pv(int_rate/12,new_mos,0,-sold_ppsf_vector)])

total_sellout_price = total_sf_vector*new_pv_ppsf_vector
#Note: you should define the BR_SF differently in future, make them toggle-able


In [72]:
new_brs = np.array([10,20,30])
sfs = np.array([400,600,800])

In [ ]:
fun = lambda x: ()

In [ ]:
fun = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2